In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import  train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import chi2
import xgboost as xgb

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

C:\Users\rwang\AppData\Local\Continuum\Miniconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
hierarchy = pd.read_csv("company_hierarchy.csv",index_col='employee_id')
hierarchy['level'] = None # initialize 

# starting point
hierarchy.loc[hierarchy.dept == 'CEO','level'] = 'CEO'

# change all boss_id to integer
hierarchy.loc[hierarchy.level == 'CEO','boss_id'] = -1
hierarchy['boss_id'] = hierarchy.boss_id.astype(int)

def set_level(boss_level,level):
    boss_ids = hierarchy.loc[hierarchy.level == boss_level,:].index
    is_subordinate = np.in1d(hierarchy.boss_id,boss_ids)
    hierarchy.loc[is_subordinate,'level'] = level

set_level('CEO','E')
set_level('E','VP')
set_level('VP','D')
set_level('D','MM')
set_level('MM','IC')

In [3]:
hierarchy.level.value_counts()

IC     9000
MM      800
D       160
VP       35
E         4
CEO       1
Name: level, dtype: int64

In [4]:
hierarchy.head()

,boss_id,dept,level
employee_id,,,
46456,175361,sales,IC
104708,29733,HR,IC
120853,41991,sales,IC
142630,171266,HR,MM
72711,198240,sales,IC


In [5]:
hierarchy['n_subordinates'] = 0 # initialize

def __count_subordinates(s):
    """
    s: stands for subordinates, it is a series
    its value represents #subordinates that subordinate has
    """
    n_direct_subordinates = s.shape[0]
    n_indirect_subordinates = s.sum()
    return n_direct_subordinates + n_indirect_subordinates

def count_subordinates(subordinate_level):
    num_subordinates = hierarchy.loc[hierarchy.level == subordinate_level,:].groupby('boss_id')['n_subordinates'].agg(__count_subordinates)
    hierarchy.loc[num_subordinates.index,'n_subordinates'] = num_subordinates

count_subordinates(subordinate_level="IC")
count_subordinates(subordinate_level="MM")
count_subordinates(subordinate_level="D")
count_subordinates(subordinate_level="VP")
count_subordinates(subordinate_level="E")

In [6]:
hierarchy.head()

,boss_id,dept,level,n_subordinates
employee_id,,,,
46456,175361,sales,IC,0
104708,29733,HR,IC,0
120853,41991,sales,IC,0
142630,171266,HR,MM,16
72711,198240,sales,IC,0


In [7]:
employees = pd.read_csv("employee.csv",index_col="employee_id")
employees["salary"] /= 1000

In [8]:

employees = employees.join(hierarchy)
employees.to_csv("all_employees.csv",index_label="employee_id")

In [9]:
employees.head()

,signing_bonus,salary,degree_level,sex,yrs_experience,boss_id,dept,level,n_subordinates
employee_id,,,,,,,,,
138719,0,273.0,Master,M,2,43602,engineering,IC,0
3192,0,301.0,Bachelor,F,1,87847,sales,IC,0
114657,0,261.0,Master,F,2,180854,sales,IC,0
29039,0,86.0,High_School,F,4,88370,HR,IC,0
118607,0,126.0,Bachelor,F,3,23565,sales,IC,0


In [10]:
employees['Gender'] = employees['sex'].map( {'F': 0, 'M': 1} ).astype(int)

In [11]:
dept = pd.get_dummies(employees['dept'],prefix = 'dept')
employees = pd.concat([employees,dept],axis = 1)


In [12]:
level = pd.get_dummies(employees['level'],prefix = 'level')
employees = pd.concat([employees,level],axis = 1)

In [13]:
degree_level = pd.get_dummies(employees['degree_level'],prefix = 'degree_level')
employees = pd.concat([employees,degree_level],axis = 1)

In [15]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 138719 to 72227
Data columns (total 25 columns):
signing_bonus               10000 non-null int64
salary                      10000 non-null float64
degree_level                10000 non-null object
sex                         10000 non-null object
yrs_experience              10000 non-null int64
boss_id                     10000 non-null int32
dept                        10000 non-null object
level                       10000 non-null object
n_subordinates              10000 non-null int64
Gender                      10000 non-null int32
dept_CEO                    10000 non-null uint8
dept_HR                     10000 non-null uint8
dept_engineering            10000 non-null uint8
dept_marketing              10000 non-null uint8
dept_sales                  10000 non-null uint8
level_CEO                   10000 non-null uint8
level_D                     10000 non-null uint8
level_E                     10000 non-null uint

In [68]:
y = employees['salary']
X = employees[['signing_bonus','yrs_experience','n_subordinates','Gender','dept_HR','dept_engineering','dept_marketing','dept_sales',
              'level_D','level_E','level_IC','level_MM','level_VP','degree_level_Bachelor','degree_level_Master','degree_level_PhD']]

In [52]:
import statsmodels.formula.api as smf
model = smf.OLS(y, X)
results = model.fit()
# Statsmodels gives R-like statistical output
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.882
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     8328.
Date:                Sat, 14 Oct 2017   Prob (F-statistic):               0.00
Time:                        21:22:39   Log-Likelihood:                -56911.
No. Observations:               10000   AIC:                         1.138e+05
Df Residuals:                    9991   BIC:                         1.139e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
n_subordinates       0.0691      0.007  

engineering dept have more salary than other depts.However, HR dept has way lower salary than others. E have much higher salary. n_subordinates contribute very little to the salary, which is not fair. 

In [48]:
results = results.pvalues.reset_index()

In [49]:
results.columns = ['feature','pvalue']

In [50]:
del X[results.loc[results['pvalue'].argmax()].tolist()[0]]

In [51]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 138719 to 72227
Data columns (total 10 columns):
n_subordinates      10000 non-null int64
dept_HR             10000 non-null uint8
dept_engineering    10000 non-null uint8
dept_marketing      10000 non-null uint8
dept_sales          10000 non-null uint8
level_D             10000 non-null uint8
level_E             10000 non-null uint8
level_IC            10000 non-null uint8
level_MM            10000 non-null uint8
level_VP            10000 non-null uint8
dtypes: int64(1), uint8(9)
memory usage: 564.1 KB


In [64]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [65]:
feature_cols = X.columns.tolist()

In [66]:
zip(feature_cols, lm.coef_)

[('n_subordinates', 0.037340337148121995),
 ('Gender', -0.82070429727607652),
 ('dept_HR', -135.77546205769562),
 ('dept_engineering', 23.556857924565168),
 ('dept_marketing', -25.52980100707245),
 ('dept_sales', -25.598774710735086),
 ('level_D', -85.852969961451706),
 ('level_E', 183.19348258361799),
 ('level_IC', -108.04520888869665),
 ('level_MM', -103.62405908849516),
 ('level_VP', -49.018424495905073)]

In [67]:
lm.score(X, y)

0.35103075207980539

In [69]:
from sklearn.linear_model import Ridge
ri = Ridge()
ri.fit(X, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [70]:
ri.score(X, y)

0.3512586158838924

In [71]:
from sklearn.linear_model import Lasso
la = Lasso()
la.fit(X, y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [72]:
la.score(X, y)

0.34691251639645349